In [1]:
import seaborn as sns
import pandas as pd
import numpy as np
import altair as alt
from markdown import markdown
from IPython.display import Markdown
from ipywidgets.widgets import HTML, Tab
from ipywidgets import widgets
from datetime import timedelta
from matplotlib import pyplot as plt
import os.path as op

from mod import load_data, alt_theme

from warnings import simplefilter
simplefilter('ignore')

In [2]:
def author_url(author):
    return f"https://github.com/{author}"

In [3]:
# Parameters
fmt_date = "{:%Y-%m-%d}"

n_days = 90
start_date = fmt_date.format(pd.datetime.today() - timedelta(days=n_days))
end_date = fmt_date.format(pd.datetime.today())

renderer = "jupyterlab"
github_orgs = ["jupyterhub", "jupyter", "jupyterlab", "jupyter-widgets", "ipython", "binder-examples", "nteract"]

In [4]:
# Parameters
renderer = "kaggle"
github_orgs = ["jupyterhub", "jupyter", "jupyterlab", "jupyter-widgets", "ipython", "binder-examples"]
start_date = "2019-08-01"
end_date = "2019-09-01"


In [5]:
comments, issues, prs = load_data('../data/')
bot_names = pd.read_csv('bot_names.csv')['names'].tolist()
comments = comments.query('author not in @bot_names').drop_duplicates()
issues = issues.query('author not in @bot_names').drop_duplicates()
prs = prs.query('author not in @bot_names').drop_duplicates()

In [6]:
# Only keep the datapoints we want
query_data = 'updatedAt > @start_date and updatedAt < @end_date and org in @github_orgs'
comments = comments.query(query_data)
issues = issues.query(query_data)
prs = prs.query(query_data)

In [7]:
alt.renderers.enable(renderer);
alt.themes.register('my_theme', alt_theme)
alt.themes.enable("my_theme")

ThemeRegistry.enable('my_theme')

In [8]:
# Information about out time window
time_delta = pd.to_datetime(end_date) - pd.to_datetime(start_date)
n_days = time_delta.days

# Information about the data we loaded
github_orgs = comments['org'].unique()

# GitHub activity

Jupyter also has lots of activity across GitHub repositories. The following sections contain
overviews of recent activity across the following GitHub organizations:

In [9]:
# Define colors we'll use for GitHub membership
author_types = ['MEMBER', 'CONTRIBUTOR', 'COLLABORATOR', "NONE"]

author_palette = sns.palettes.blend_palette(["lightgrey", "lightgreen", "darkgreen"], 4)
author_colors = ["rgb({}, {}, {}, {})".format(*(ii*256)) for ii in author_palette]
author_color_dict = {key: val for key, val in zip(author_types, author_palette)}

In [10]:
orgs_md = []
for org in github_orgs:
    orgs_md.append(f'* [github.com/{org}](https://github.com/{org})')
Markdown('\n'.join(orgs_md))

* [github.com/binder-examples](https://github.com/binder-examples)
* [github.com/ipython](https://github.com/ipython)
* [github.com/jupyter](https://github.com/jupyter)
* [github.com/jupyter-widgets](https://github.com/jupyter-widgets)
* [github.com/jupyterhub](https://github.com/jupyterhub)
* [github.com/jupyterlab](https://github.com/jupyterlab)

In [11]:
Markdown(f"Showing GitHub activity from **{start_date}** to **{end_date}**")

Showing GitHub activity from **2019-08-01** to **2019-09-01**

## List of all contributors per organization

First, we'll list each contributor that has contributed to each organization in the last several days.
Contributions to open source projects are diverse, and involve much more than just contributing code and
code review. Thanks to everybody in the Jupyter communities for all that they do.

In [12]:
n_plot = 5
tabs = widgets.Tab(children=[])
for ii, org in enumerate(github_orgs):
    authors_comments = comments.query('org == @org')['author']
    authors_prs = prs.query('org == @org')['author']
    unique_participants = np.unique(np.hstack([authors_comments.values, authors_prs.values]).astype(str)).tolist()
    unique_participants.sort(key=lambda a: a.lower())
    
    all_participants = [f"[{participant}](https://github.com/{participant})" for participant in unique_participants]
    participants_md = " | ".join(all_participants)
    md_html = HTML("<center>{}</center>".format(markdown(participants_md)))
    
    children = list(tabs.children)
    children.append(md_html)
    tabs.children = tuple(children)
    tabs.set_title(ii, org)
    
display(Markdown(f"All participants across issues and pull requests in each org in the last {n_days} days"))
display(tabs)

All participants across issues and pull requests in each org in the last 31 days

## Merged Pull requests

Here's an analysis of **merged pull requests** across each of the repositories in the Jupyter
ecosystem.

In [13]:
merged = prs.query('state == "MERGED" and closedAt > @start_date and closedAt < @end_date')

In [14]:
prs_by_repo = merged.groupby(['org', 'repo']).count()['author'].reset_index().sort_values(['org', 'author'], ascending=False)
alt.Chart(data=prs_by_repo, title=f"Merged PRs in the last {n_days} days").mark_bar().encode(
    x=alt.X('repo', sort=prs_by_repo['repo'].values.tolist()),
    y='author',
    color='org'
)

Chart({
  data:                 org                         repo  author
  35       jupyterlab                   jupyterlab      26
  33       jupyterlab                     debugger      11
  41       jupyterlab            jupyterlab_server       6
  42       jupyterlab                pull-requests       5
  37       jupyterlab     jupyterlab-data-explorer       4
  38       jupyterlab               jupyterlab-git       3
  34       jupyterlab            jupyter-renderers       1
  36       jupyterlab        jupyterlab-commenting       1
  39       jupyterlab              jupyterlab-hdf5       1
  40       jupyterlab  jupyterlab-metadata-service       1
  24       jupyterhub                    binderhub      12
  26       jupyterhub                   jupyterhub       7
  32       jupyterhub       zero-to-jupyterhub-k8s       7
  27       jupyterhub          mybinder.org-deploy       6
  28       jupyterhub                  nbgitpuller       2
  23       jupyterhub                       binder       1
  25       jupyterhub        firstuseauthenticator       1
  29       jupyterhub               oauthenticator       1
  30       jupyterhub                 team-compass       1
  31       jupyterhub                traefik-proxy       1
  21  jupyter-widgets                   ipywidgets       2
  22  jupyter-widgets          widget-cookiecutter       1
  14          jupyter                     nbgrader      15
  7           jupyter                 jupyter-book      13
  6           jupyter                      jupyter       9
  17          jupyter                    qtconsole       7
  5           jupyter           enterprise_gateway       5
  4           jupyter                docker-stacks       4
  10          jupyter               jupyter_server       4
  11          jupyter               kernel_gateway       4
  12          jupyter                    nbconvert       4
  18          jupyter                  repo2docker       4
  16          jupyter                     notebook       2
  3           jupyter                       design       1
  8           jupyter            jupyter.github.io       1
  9           jupyter               jupyter_client       1
  13          jupyter                       nbdime       1
  15          jupyter                     nbviewer       1
  19          jupyter                    telemetry       1
  20          jupyter                     testpath       1
  0           ipython                    ipykernel       4
  1           ipython                      ipython       2
  2           ipython                    traitlets       2,
  encoding: FacetedEncoding({
    color: Color({
      field: 'org',
      type: 'nominal'
    }),
    x: X({
      field: 'repo',
      sort: ['jupyterlab', 'debugger', 'jupyterlab_server', 'pull-requests', 'jupyterlab-data-explorer', 'jupyterlab-git', 'jupyter-renderers', 'jupyterlab-commenting', 'jupyterlab-hdf5', 'jupyterlab-metadata-service', 'binderhub', 'jupyterhub', 'zero-to-jupyterhub-k8s', 'mybinder.org-deploy', 'nbgitpuller', 'binder', 'firstuseauthenticator', 'oauthenticator', 'team-compass', 'traefik-proxy', 'ipywidgets', 'widget-cookiecutter', 'nbgrader', 'jupyter-book', 'jupyter', 'qtconsole', 'enterprise_gateway', 'docker-stacks', 'jupyter_server', 'kernel_gateway', 'nbconvert', 'repo2docker', 'notebook', 'design', 'jupyter.github.io', 'jupyter_client', 'nbdime', 'nbviewer', 'telemetry', 'testpath', 'ipykernel', 'ipython', 'traitlets'],
      type: 'nominal'
    }),
    y: Y({
      field: 'author',
      type: 'quantitative'
    })
  }),
  mark: 'bar',
  title: 'Merged PRs in the last 31 days'
})

### A list of merged PRs by project

Below is a tabbed readout of recently-merged PRs. Check out the title to get an idea for what they
implemented, and be sure to thank the PR author for their hard work!

In [15]:
tabs = widgets.Tab(children=[])
merged_by = {}
pr_by = {}
for ii, (org, idata) in enumerate(merged.groupby('org')):
    issue_md = []
    issue_md.append(f"#### Closed PRs for org: `{org}`")
    issue_md.append("")
    for (org, repo), prs in idata.groupby(['org', 'repo']):
        issue_md.append(f"##### [{org}/{repo}](https://github.com/{org}/{repo})")
        for _, pr in prs.iterrows():
            user_name = pr['author']
            user_url = author_url(user_name)
            pr_number = pr['number']
            pr_html = pr['url']
            pr_title = pr['title']
            pr_closedby = pr['mergedBy']
            pr_closedby_url = f"https://github.com/{pr_closedby}"
            if user_name not in pr_by:
                pr_by[user_name] = 1
            else:
                pr_by[user_name] += 1
                
            if pr_closedby not in merged_by:
                merged_by[pr_closedby] = 1
            else:
                merged_by[pr_closedby] += 1
            text = f"* [(#{pr_number})]({pr_html}): _{pr_title}_ by **[@{user_name}]({user_url})** merged by **[@{pr_closedby}]({pr_closedby_url})**"
            issue_md.append(text)
    issue_md.append('')
    markdown_html = markdown('\n'.join(issue_md))
    
    children = list(tabs.children)
    children.append(HTML(markdown_html))
    tabs.children = tuple(children)
    tabs.set_title(ii, org)
tabs

### Authoring and merging stats by repository

Let's see who has been doing most of the PR authoring and merging. The PR author is generally the
person that implemented a change in the repository (code, documentation, etc). The PR merger is
the person that "pressed the green button" and got the change into the main codebase.

In [16]:
# Prep our merging DF
merged_by_repo = merged.groupby(['org', 'repo', 'author'], as_index=False).agg({'id': 'count', 'authorAssociation': 'first'}).rename(columns={'id': "authored", 'author': 'username'})
closed_by_repo = merged.groupby(['org', 'repo', 'mergedBy']).count()['id'].reset_index().rename(columns={'id': "closed", "mergedBy": "username"})

In [17]:
n_plot = 50
charts = []
for ii, (iorg, idata) in enumerate(merged_by_repo.replace(np.nan, 0).groupby(['org'])):
    title = f"PR authors for {iorg} in the last {n_days} days"
    idata = idata.groupby('username', as_index=False).agg({'authored': 'sum', 'authorAssociation': 'first'})
    idata = idata.sort_values('authored', ascending=False).head(n_plot)
    ch = alt.Chart(data=idata, width=1000, title=title).mark_bar().encode(
        x='username',
        y='authored',
        color=alt.Color('authorAssociation', scale=alt.Scale(domain=author_types, range=author_colors))
    )
    charts.append(ch)

alt.hconcat(*charts)

HConcatChart({
  hconcat: [Chart({
    data:         username  authored authorAssociation
    1   danielballan         2       CONTRIBUTOR
    0  SylvainCorlay         1            MEMBER
    2  edisongustavo         1       CONTRIBUTOR
    3       krinsman         1       CONTRIBUTOR
    4          minrk         1            MEMBER
    5      terrdavis         1       CONTRIBUTOR
    6       willingc         1            MEMBER,
    encoding: FacetedEncoding({
      color: Color({
        field: 'authorAssociation',
        scale: Scale({
          domain: ['MEMBER', 'CONTRIBUTOR', 'COLLABORATOR', 'NONE'],
          range: ['rgb(211.82745098039214, 211.82745098039214, 211.82745098039214, 256.0)', 'rgb(166.98562091503268, 229.89803921568628, 166.98562091503268, 256.0)', 'rgb(96.37647058823529, 192.75294117647059, 96.37647058823529, 256.0)', 'rgb(0.0, 100.3921568627451, 0.0, 256.0)']
        }),
        type: 'nominal'
      }),
      x: X({
        field: 'username',
        type: 'nominal'
      }),
      y: Y({
        field: 'authored',
        type: 'quantitative'
      })
    }),
    mark: 'bar',
    title: 'PR authors for ipython in the last 31 days',
    width: 1000
  }), Chart({
    data:                  username  authored authorAssociation
    9               choldgraf        12            MEMBER
    17               jhamrick        11            MEMBER
    15               impact27         6       CONTRIBUTOR
    19            kevin-bates         5            MEMBER
    25                rkdarst         5       CONTRIBUTOR
    24                parente         4            MEMBER
    20               lresende         4            MEMBER
    5                 Zsailer         2            MEMBER
    29  transifex-integration         2       CONTRIBUTOR
    22                  minrk         2            MEMBER
    14              gjenkins8         2       CONTRIBUTOR
    21          martinagvilas         2       CONTRIBUTOR
    27               t-makaro         1      COLLABORATOR
    26                 royopa         1       CONTRIBUTOR
    28              takluyver         1            MEMBER
    23       nareshsankapelly         1       CONTRIBUTOR
    30        tv-watts-tatari         1       CONTRIBUTOR
    31               voyageth         1       CONTRIBUTOR
    0            AartGoossens         1       CONTRIBUTOR
    16          jeffyjefflabs         1       CONTRIBUTOR
    18        juliusvonkohout         1       CONTRIBUTOR
    1             Geektrovert         1       CONTRIBUTOR
    13                  dniku         1       CONTRIBUTOR
    12             dm-wyncode         1       CONTRIBUTOR
    11            davidrobles         1       CONTRIBUTOR
    10           consideRatio         1       CONTRIBUTOR
    8              ccordoba12         1      COLLABORATOR
    7                 betatim         1            MEMBER
    6                  arnavs         1       CONTRIBUTOR
    4                 Sefriol         1       CONTRIBUTOR
    3                   MSeal         1            MEMBER
    2          GeorgianaElena         1       CONTRIBUTOR
    32           williamstein         1       CONTRIBUTOR,
    encoding: FacetedEncoding({
      color: Color({
        field: 'authorAssociation',
        scale: Scale({
          domain: ['MEMBER', 'CONTRIBUTOR', 'COLLABORATOR', 'NONE'],
          range: ['rgb(211.82745098039214, 211.82745098039214, 211.82745098039214, 256.0)', 'rgb(166.98562091503268, 229.89803921568628, 166.98562091503268, 256.0)', 'rgb(96.37647058823529, 192.75294117647059, 96.37647058823529, 256.0)', 'rgb(0.0, 100.3921568627451, 0.0, 256.0)']
        }),
        type: 'nominal'
      }),
      x: X({
        field: 'username',
        type: 'nominal'
      }),
      y: Y({
        field: 'authored',
        type: 'quantitative'
      })
    }),
    mark: 'bar',
    title: 'PR authors for jupyter in the last 31 days',
    width: 1000
  }), Chart({
    data:        username  authored authorA

In [18]:
charts = []
for ii, (iorg, idata) in enumerate(closed_by_repo.replace(np.nan, 0).groupby(['org'])):
    title = f"Merges for {iorg} in the last {n_days} days"
    ch = alt.Chart(data=idata, width=1000, title=title).mark_bar().encode(
        x='username',
        y='closed',
    )
    charts.append(ch)
alt.hconcat(*charts)

HConcatChart({
  hconcat: [Chart({
    data:        org       repo       username  closed
    0  ipython  ipykernel  SylvainCorlay       1
    1  ipython  ipykernel      blink1073       3
    2  ipython    ipython        Carreau       2
    3  ipython  traitlets       rmorshea       2,
    encoding: FacetedEncoding({
      x: X({
        field: 'username',
        type: 'nominal'
      }),
      y: Y({
        field: 'closed',
        type: 'quantitative'
      })
    }),
    mark: 'bar',
    title: 'Merges for ipython in the last 31 days',
    width: 1000
  }), Chart({
    data:         org                repo     username  closed
    4   jupyter              design    choldgraf       1
    5   jupyter       docker-stacks      parente       4
    6   jupyter  enterprise_gateway  kevin-bates       4
    7   jupyter  enterprise_gateway     lresende       1
    8   jupyter             jupyter    choldgraf       3
    9   jupyter             jupyter      parente       6
    10  jupyter        jupyter-book    choldgraf      13
    11  jupyter   jupyter.github.io    choldgraf       1
    12  jupyter      jupyter_client        MSeal       1
    13  jupyter      jupyter_server      Zsailer       3
    14  jupyter      jupyter_server    blink1073       1
    15  jupyter      kernel_gateway     lresende       4
    16  jupyter           nbconvert        MSeal       3
    17  jupyter           nbconvert     t-makaro       1
    18  jupyter              nbdime      vidartf       1
    19  jupyter            nbgrader     jhamrick      15
    20  jupyter            nbviewer      parente       1
    21  jupyter            notebook     lresende       2
    22  jupyter           qtconsole   ccordoba12       7
    23  jupyter         repo2docker      betatim       4
    24  jupyter           telemetry   jaipreet-s       1
    25  jupyter            testpath    blink1073       1,
    encoding: FacetedEncoding({
      x: X({
        field: 'username',
        type: 'nominal'
      }),
      y: Y({
        field: 'closed',
        type: 'quantitative'
      })
    }),
    mark: 'bar',
    title: 'Merges for jupyter in the last 31 days',
    width: 1000
  }), Chart({
    data:                 org                 repo         username  closed
    26  jupyter-widgets           ipywidgets       jasongrout       1
    27  jupyter-widgets           ipywidgets  maartenbreddels       1
    28  jupyter-widgets  widget-cookiecutter       jasongrout       1,
    encoding: FacetedEncoding({
      x: X({
        field: 'username',
        type: 'nominal'
      }),
      y: Y({
        field: 'closed',
        type: 'quantitative'
      })
    }),
    mark: 'bar',
    title: 'Merges for jupyter-widgets in the last 31 days',
    width: 1000
  }), Chart({
    data:            org                    repo        username  closed
    29  jupyterhub                  binder         betatim       1
    30  jupyterhub               binderhub         betatim      10
    31  jupyterhub               binderhub       choldgraf       2
    32  jupyterhub   firstuseauthenticator  GeorgianaElena       1
    33  jupyterhub              jupyterhub  GeorgianaElena       2
    34  jupyterhub              jupyterhub         betatim       4
    35  jupyterhub              jupyterhub           minrk       1
    36  jupyterhub     mybinder.org-deploy         betatim       6
    37  jupyterhub             nbgitpuller       yuvipanda       2
    38  jupyterhub          oauthenticator           minrk       1
    39  jupyterhub            team-compass       choldgraf       1
    40  jupyterhub           traefik-proxy  GeorgianaElena       1
    41  jupyterhub  zero-to-jupyterhub-k8s         betatim       4
    42  jupyterhub  zero-to-jupyterhub-k8s    consideRatio       3,
    encoding: FacetedEncoding({
      x: X({
        field: 'username',
        type: 'nominal'
      }),
      y: Y({
        field: 'closed',
        type: 'quantitative'
      })
    }),
    mark: 'bar',
    title: 

## Issues

Issues are **conversations** that happen on our GitHub repositories. Here's an
analysis of issues across the Jupyter organizations.

In [19]:
created = issues.query('state == "OPEN" and createdAt > @start_date and createdAt < @end_date')
closed = issues.query('state == "CLOSED" and closedAt > @start_date and closedAt < @end_date')

In [20]:
created_counts = created.groupby(['org', 'repo']).count()['number'].reset_index()
created_counts['org/repo'] = created_counts.apply(lambda a: a['org'] + '/' + a['repo'], axis=1)
sorted_vals = created_counts.sort_values(['org', 'number'], ascending=False)['repo'].values
alt.Chart(data=created_counts, title=f"Issues created in the last {n_days} days").mark_bar().encode(
    x=alt.X('repo', sort=alt.Sort(sorted_vals.tolist())),
    y='number',
    color='org',
)

Chart({
  data:                 org                        repo  number  \
  0           ipython                 ipyparallel       1   
  1           ipython                     ipython      11   
  2           jupyter               accessibility       1   
  3           jupyter               atom-notebook       1   
  4           jupyter               docker-stacks       3   
  5           jupyter          enterprise_gateway       1   
  6           jupyter                     jupyter       2   
  7           jupyter                jupyter-book       3   
  8           jupyter              jupyter_client       1   
  9           jupyter                jupyter_core       2   
  10          jupyter              jupyter_server       1   
  11          jupyter                   nbconvert       3   
  12          jupyter                      nbdime       2   
  13          jupyter                    nbgrader       5   
  14          jupyter                    nbviewer       1   
  15          jupyter                    notebook      42   
  16          jupyter                   qtconsole       2   
  17          jupyter                 repo2docker       2   
  18          jupyter                   telemetry       3   
  19  jupyter-widgets                  ipyleaflet       2   
  20  jupyter-widgets                  ipywidgets       6   
  21  jupyter-widgets          jupyterlab-sidecar       3   
  22  jupyter-widgets                   pythreejs       1   
  23  jupyter-widgets                  traittypes       1   
  24       jupyterhub                batchspawner       1   
  25       jupyterhub                   binderhub       1   
  26       jupyterhub               dockerspawner       4   
  27       jupyterhub       firstuseauthenticator       1   
  28       jupyterhub        jupyter-server-proxy       1   
  29       jupyterhub                  jupyterhub      11   
  30       jupyterhub  jupyterhub-deploy-teaching       1   
  31       jupyterhub                 kubespawner       2   
  32       jupyterhub           ldapauthenticator       4   
  33       jupyterhub         mybinder.org-deploy       1   
  34       jupyterhub                 nbgitpuller       1   
  35       jupyterhub              oauthenticator       3   
  36       jupyterhub                 sudospawner       1   
  37       jupyterhub                team-compass       4   
  38       jupyterhub     the-littlest-jupyterhub       5   
  39       jupyterhub               traefik-proxy       1   
  40       jupyterhub      zero-to-jupyterhub-k8s      14   
  41       jupyterlab                    debugger       1   
  42       jupyterlab                  jupyterlab      34   
  43       jupyterlab         jupyterlab-celltags       1   
  44       jupyterlab       jupyterlab-commenting       3   
  45       jupyterlab    jupyterlab-data-explorer      10   
  46       jupyterlab              jupyterlab-git       3   
  47       jupyterlab           jupyterlab-github       1   
  48       jupyterlab             jupyterlab-hdf5       7   
  49       jupyterlab            jupyterlab-latex       2   
  50       jupyterlab              jupyterlab-toc       2   
  51       jupyterlab               pull-requests       2   
  
                                   org/repo  
  0                     ipython/ipyparallel  
  1                         ipython/ipython  
  2                   jupyter/accessibility  
  3                   jupyter/atom-notebook  
  4                   jupyter/docker-stacks  
  5              jupyter/enterprise_gateway  
  6                         jupyter/jupyter  
  7                    jupyter/jupyter-book  
  8                  jupyter/jupyter_client  
  9                    jupyter/jupyter_core  
  10                 jupyter/jupyter_server  
  11                      jupyter/nbconvert  
  12                         jupyter/nbdime  
  13                       jupyter/nbgrader  
  14                       jupyter/nbviewer  
  15          

In [21]:
closed_counts = closed.groupby(['org', 'repo']).count()['number'].reset_index()
closed_counts['org/repo'] = closed_counts.apply(lambda a: a['org'] + '/' + a['repo'], axis=1)
sorted_vals = closed_counts.sort_values(['org', 'number'], ascending=False)['repo'].values
alt.Chart(data=closed_counts, title=f"Issues closed in the last {n_days} days").mark_bar().encode(
    x=alt.X('repo', sort=alt.Sort(sorted_vals.tolist())),
    y='number',
    color='org',
)

Chart({
  data:                 org                      repo  number  \
  0           ipython                 ipykernel       3   
  1           ipython                   ipython      12   
  2           jupyter                    design       1   
  3           jupyter             docker-stacks       8   
  4           jupyter        enterprise_gateway       2   
  5           jupyter              jupyter-book       7   
  6           jupyter            jupyter-sphinx       1   
  7           jupyter            jupyter_client       1   
  8           jupyter            jupyter_server       2   
  9           jupyter                 nbconvert      50   
  10          jupyter                    nbdime       7   
  11          jupyter                  nbgrader      21   
  12          jupyter                  nbviewer       2   
  13          jupyter                  notebook       7   
  14          jupyter                 qtconsole       4   
  15          jupyter               repo2docker       7   
  16  jupyter-widgets                ipyleaflet       4   
  17  jupyter-widgets                ipywidgets       6   
  18  jupyter-widgets    widget-ts-cookiecutter       1   
  19       jupyterhub                 binderhub      11   
  20       jupyterhub                helm-chart       1   
  21       jupyterhub    jupyter-rsession-proxy       2   
  22       jupyterhub      jupyter-server-proxy       1   
  23       jupyterhub                jupyterhub      20   
  24       jupyterhub               kubespawner       4   
  25       jupyterhub       mybinder.org-deploy       9   
  26       jupyterhub       nativeauthenticator       2   
  27       jupyterhub            oauthenticator       1   
  28       jupyterhub              team-compass       2   
  29       jupyterhub   the-littlest-jupyterhub       5   
  30       jupyterhub             traefik-proxy       2   
  31       jupyterhub    zero-to-jupyterhub-k8s      10   
  32       jupyterlab                  debugger      10   
  33       jupyterlab         jupyter-renderers       1   
  34       jupyterlab                jupyterlab       1   
  35       jupyterlab       jupyterlab-celltags       1   
  36       jupyterlab  jupyterlab-data-explorer       8   
  37       jupyterlab            jupyterlab-git       3   
  38       jupyterlab           jupyterlab-hdf5       2   
  39       jupyterlab             pull-requests       2   
  
                                    org/repo  
  0                        ipython/ipykernel  
  1                          ipython/ipython  
  2                           jupyter/design  
  3                    jupyter/docker-stacks  
  4               jupyter/enterprise_gateway  
  5                     jupyter/jupyter-book  
  6                   jupyter/jupyter-sphinx  
  7                   jupyter/jupyter_client  
  8                   jupyter/jupyter_server  
  9                        jupyter/nbconvert  
  10                          jupyter/nbdime  
  11                        jupyter/nbgrader  
  12                        jupyter/nbviewer  
  13                        jupyter/notebook  
  14                       jupyter/qtconsole  
  15                     jupyter/repo2docker  
  16              jupyter-widgets/ipyleaflet  
  17              jupyter-widgets/ipywidgets  
  18  jupyter-widgets/widget-ts-cookiecutter  
  19                    jupyterhub/binderhub  
  20                   jupyterhub/helm-chart  
  21       jupyterhub/jupyter-rsession-proxy  
  22         jupyterhub/jupyter-server-proxy  
  23                   jupyterhub/jupyterhub  
  24                  jupyterhub/kubespawner  
  25          jupyterhub/mybinder.org-deploy  
  26          jupyterhub/nativeauthenticator  
  27               jupyterhub/oauthenticator  
  28                 jupyterhub/team-compass  
  29      jupyterhub/the-littlest-jupyterhub  
  30                jupyterhub/traefik-proxy  
  31       jupyterhub/zero-to-jupyterhub-k8s  
  32         

In [22]:
created_closed = pd.merge(created_counts.rename(columns={'number': 'created'}).drop(columns='org/repo'),
                          closed_counts.rename(columns={'number': 'closed'}).drop(columns='org/repo'),
                          on=['org', 'repo'], how='outer')

created_closed = pd.melt(created_closed, id_vars=['org', 'repo'], var_name="kind", value_name="count").replace(np.nan, 0)

In [23]:
charts = []
for org in github_orgs:
    # Pick the top 10 repositories
    this_issues = created_closed.query('org == @org')
    top_repos = this_issues.groupby(['repo']).sum().sort_values(by='count', ascending=False).head(10).index
    ch = alt.Chart(this_issues.query('repo in @top_repos'), width=120).mark_bar().encode(
        x=alt.X("kind", axis=alt.Axis(labelFontSize=15, title="")), 
        y=alt.Y('count', axis=alt.Axis(titleFontSize=15, labelFontSize=12)),
        color='kind',
        column=alt.Column("repo", header=alt.Header(title=f"Issue activity, last {n_days} days for {org}", titleFontSize=15, labelFontSize=12))
    )
    charts.append(ch)
alt.hconcat(*charts)

HConcatChart({
  hconcat: [Chart({
    data: Empty DataFrame
    Columns: [org, repo, kind, count]
    Index: [],
    encoding: FacetedEncoding({
      color: Color({
        field: 'kind',
        type: 'nominal'
      }),
      column: Column({
        field: 'repo',
        header: Header({
          labelFontSize: 12,
          title: 'Issue activity, last 31 days for binder-examples',
          titleFontSize: 15
        }),
        type: 'nominal'
      }),
      x: X({
        axis: Axis({
          labelFontSize: 15,
          title: ''
        }),
        field: 'kind',
        type: 'nominal'
      }),
      y: Y({
        axis: Axis({
          labelFontSize: 12,
          titleFontSize: 15
        }),
        field: 'count',
        type: 'quantitative'
      })
    }),
    mark: 'bar',
    width: 120
  }), Chart({
    data:          org         repo     kind  count
    0    ipython  ipyparallel  created    1.0
    1    ipython      ipython  created   11.0
    52   ipython    ipykernel  created    0.0
    60   ipython  ipyparallel   closed    0.0
    61   ipython      ipython   closed   12.0
    112  ipython    ipykernel   closed    3.0,
    encoding: FacetedEncoding({
      color: Color({
        field: 'kind',
        type: 'nominal'
      }),
      column: Column({
        field: 'repo',
        header: Header({
          labelFontSize: 12,
          title: 'Issue activity, last 31 days for ipython',
          titleFontSize: 15
        }),
        type: 'nominal'
      }),
      x: X({
        axis: Axis({
          labelFontSize: 15,
          title: ''
        }),
        field: 'kind',
        type: 'nominal'
      }),
      y: Y({
        axis: Axis({
          labelFontSize: 12,
          titleFontSize: 15
        }),
        field: 'count',
        type: 'quantitative'
      })
    }),
    mark: 'bar',
    width: 120
  }), Chart({
    data:         org                repo     kind  count
    4   jupyter       docker-stacks  created    3.0
    5   jupyter  enterprise_gateway  created    1.0
    7   jupyter        jupyter-book  created    3.0
    11  jupyter           nbconvert  created    3.0
    12  jupyter              nbdime  created    2.0
    13  jupyter            nbgrader  created    5.0
    15  jupyter            notebook  created   42.0
    16  jupyter           qtconsole  created    2.0
    17  jupyter         repo2docker  created    2.0
    18  jupyter           telemetry  created    3.0
    64  jupyter       docker-stacks   closed    8.0
    65  jupyter  enterprise_gateway   closed    2.0
    67  jupyter        jupyter-book   closed    7.0
    71  jupyter           nbconvert   closed   50.0
    72  jupyter              nbdime   closed    7.0
    73  jupyter            nbgrader   closed   21.0
    75  jupyter            notebook   closed    7.0
    76  jupyter           qtconsole   closed    4.0
    77  jupyter         repo2docker   closed    7.0
    78  jupyter           telemetry   closed    0.0,
    encoding: FacetedEncoding({
      color: Color({
        field: 'kind',
        type: 'nominal'
      }),
      column: Column({
        field: 'repo',
        header: Header({
          labelFontSize: 12,
          title: 'Issue activity, last 31 days for jupyter',
          titleFontSize: 15
        }),
        type: 'nominal'
      }),
      x: X({
        axis: Axis({
          labelFontSize: 15,
          title: ''
        }),
        field: 'kind',
        type: 'nominal'
      }),
      y: Y({
        axis: Axis({
          labelFontSize: 12,
          titleFontSize: 15
        }),
        field: 'count',
        type: 'quantitative'
      })
    }),
    mark: 'bar',
    width: 120
  }), Chart({
    data:                  org                    repo     kind  count
    19   jupyter-widgets              ipyleaflet  created    2.0
    20   jupyter-widgets              ipywidgets  created    6.0
    21   jupyter-widgets      jupyterlab-sidecar  created    3.0
    22   jupyter-widgets            

In [24]:
# Set to datetime
for kind in ['createdAt', 'closedAt']:
    closed.loc[:, kind] = pd.to_datetime(closed[kind])
    
closed.loc[:, 'time_open'] = closed['closedAt'] - closed['createdAt']
closed.loc[:, 'time_open'] = closed['time_open'].dt.total_seconds()

In [25]:
time_open = closed.groupby(['org', 'repo']).agg({'time_open': 'median'}).reset_index()
time_open['time_open'] = time_open['time_open'] / (60 * 60 * 24)
time_open['org/repo'] = time_open.apply(lambda a: a['org'] + '/' + a['repo'], axis=1)
sorted_vals = time_open.sort_values(['org', 'time_open'], ascending=False)['repo'].values
alt.Chart(data=time_open, title=f"Time to close for issues closed in the last {n_days} days").mark_bar().encode(
    x=alt.X('repo', sort=alt.Sort(sorted_vals.tolist())),
    y=alt.Y('time_open', title="Median Days Open"),
    color='org',
)

Chart({
  data:                 org                      repo   time_open  \
  0           ipython                 ipykernel    0.321123   
  1           ipython                   ipython    1.450625   
  2           jupyter                    design   72.712697   
  3           jupyter             docker-stacks   12.624913   
  4           jupyter        enterprise_gateway   12.143825   
  5           jupyter              jupyter-book   26.282049   
  6           jupyter            jupyter-sphinx   22.005833   
  7           jupyter            jupyter_client    1.214792   
  8           jupyter            jupyter_server    9.630226   
  9           jupyter                 nbconvert  497.632344   
  10          jupyter                    nbdime   52.451030   
  11          jupyter                  nbgrader   55.029676   
  12          jupyter                  nbviewer   30.792523   
  13          jupyter                  notebook  100.726968   
  14          jupyter                 qtconsole    0.919120   
  15          jupyter               repo2docker   14.390278   
  16  jupyter-widgets                ipyleaflet    0.827593   
  17  jupyter-widgets                ipywidgets    2.623484   
  18  jupyter-widgets    widget-ts-cookiecutter  227.856007   
  19       jupyterhub                 binderhub   64.082662   
  20       jupyterhub                helm-chart    1.136123   
  21       jupyterhub    jupyter-rsession-proxy    1.144873   
  22       jupyterhub      jupyter-server-proxy    0.011285   
  23       jupyterhub                jupyterhub  114.433478   
  24       jupyterhub               kubespawner    0.877431   
  25       jupyterhub       mybinder.org-deploy    9.327222   
  26       jupyterhub       nativeauthenticator   70.899838   
  27       jupyterhub            oauthenticator  335.810301   
  28       jupyterhub              team-compass   34.303079   
  29       jupyterhub   the-littlest-jupyterhub    0.470833   
  30       jupyterhub             traefik-proxy    4.765220   
  31       jupyterhub    zero-to-jupyterhub-k8s    0.970313   
  32       jupyterlab                  debugger    2.009444   
  33       jupyterlab         jupyter-renderers    0.645463   
  34       jupyterlab                jupyterlab    0.099502   
  35       jupyterlab       jupyterlab-celltags   50.762025   
  36       jupyterlab  jupyterlab-data-explorer   15.656175   
  37       jupyterlab            jupyterlab-git    7.457963   
  38       jupyterlab           jupyterlab-hdf5    0.592269   
  39       jupyterlab             pull-requests    0.028449   
  
                                    org/repo  
  0                        ipython/ipykernel  
  1                          ipython/ipython  
  2                           jupyter/design  
  3                    jupyter/docker-stacks  
  4               jupyter/enterprise_gateway  
  5                     jupyter/jupyter-book  
  6                   jupyter/jupyter-sphinx  
  7                   jupyter/jupyter_client  
  8                   jupyter/jupyter_server  
  9                        jupyter/nbconvert  
  10                          jupyter/nbdime  
  11                        jupyter/nbgrader  
  12                        jupyter/nbviewer  
  13                        jupyter/notebook  
  14                       jupyter/qtconsole  
  15                     jupyter/repo2docker  
  16              jupyter-widgets/ipyleaflet  
  17              jupyter-widgets/ipywidgets  
  18  jupyter-widgets/widget-ts-cookiecutter  
  19                    jupyterhub/binderhub  
  20                   jupyterhub/helm-chart  
  21       jupyterhub/jupyter-rsession-proxy  
  22         jupyterhub/jupyter-server-proxy  
  23                   jupyterhub/jupyterhub  
  24                  jupyterhub/kubespawner  
  25          jupyterhub/mybinder.org-deploy  
  26          jupyterhub/nativeauthenticator  
  27               jupyterhub/oauthenticator  
  28                 jupyterhub/team-

### A list of recent issues

Below is a list of issues with recent activity in each repository. If they seem of interest
to you, click on their links and jump in to participate!

In [26]:
# Add comment count data to issues and PRs
comment_counts = (
    comments
    .query("createdAt > @start_date and createdAt < @end_date")
    .groupby(['org', 'repo', 'issue_id'])
    .count().iloc[:, 0].to_frame()
)
comment_counts.columns = ['n_comments']
comment_counts = comment_counts.reset_index()

In [27]:
n_plot = 5
tabs = widgets.Tab(children=[])
for ii, (org, idata) in enumerate(comment_counts.groupby('org')):
    issue_md = []
    issue_md.append(f"#### {org}")
    issue_md.append("")
    for repo, i_issues in idata.groupby('repo'):
        issue_md.append(f"##### [{org}/{repo}](https://github.com/{org}/{repo})")

        top_issues = i_issues.sort_values('n_comments', ascending=False).head(n_plot)
        top_issue_list = pd.merge(issues, top_issues, left_on=['org', 'repo', 'number'], right_on=['org', 'repo', 'issue_id'])
        for _, issue in top_issue_list.sort_values('n_comments', ascending=False).head(n_plot).iterrows():
            user_name = issue['author']
            user_url = author_url(user_name)
            issue_number = issue['number']
            issue_html = issue['url']
            issue_title = issue['title']

            text = f"* [(#{issue_number})]({issue_html}): _{issue_title}_ by **[@{user_name}]({user_url})**"
            issue_md.append(text)

    issue_md.append('')
    md_html = HTML(markdown('\n'.join(issue_md)))
    
    children = list(tabs.children)
    children.append(HTML(markdown('\n'.join(issue_md))))
    tabs.children = tuple(children)
    tabs.set_title(ii, org)
    
display(Markdown(f"Here are the top {n_plot} active issues in each repository in the last {n_days} days"))
display(tabs)

Here are the top 5 active issues in each repository in the last 31 days

## Commenters across repositories

These are commenters across all issues and pull requests in the last several days.
These are colored by the commenter's association with the organization. For information
about what these associations mean, [see this StackOverflow post](https://stackoverflow.com/a/28866914/1927102).

In [28]:
commentors = (
    comments
    .query("createdAt > @start_date and createdAt < @end_date")
    .groupby(['org', 'repo', 'author', 'authorAssociation'])
    .count().rename(columns={'id': 'count'})['count']
    .reset_index()
    .sort_values(['org', 'count'], ascending=False)
)

In [29]:
n_plot = 50
charts = []
for ii, (iorg, idata) in enumerate(commentors.groupby(['org'])):
    title = f"Top {n_plot} commentors for {iorg} in the last {n_days} days"
    idata = idata.groupby('author', as_index=False).agg({'count': 'sum', 'authorAssociation': 'first'})
    idata = idata.sort_values('count', ascending=False).head(n_plot)
    ch = alt.Chart(data=idata.head(n_plot), width=1000, title=title).mark_bar().encode(
        x='author',
        y='count',
        color=alt.Color('authorAssociation', scale=alt.Scale(domain=author_types, range=author_colors))
    )
    charts.append(ch)
alt.hconcat(*charts)

HConcatChart({
  hconcat: [Chart({
    data:       author  count authorAssociation
    0  choldgraf      1            MEMBER,
    encoding: FacetedEncoding({
      color: Color({
        field: 'authorAssociation',
        scale: Scale({
          domain: ['MEMBER', 'CONTRIBUTOR', 'COLLABORATOR', 'NONE'],
          range: ['rgb(211.82745098039214, 211.82745098039214, 211.82745098039214, 256.0)', 'rgb(166.98562091503268, 229.89803921568628, 166.98562091503268, 256.0)', 'rgb(96.37647058823529, 192.75294117647059, 96.37647058823529, 256.0)', 'rgb(0.0, 100.3921568627451, 0.0, 256.0)']
        }),
        type: 'nominal'
      }),
      x: X({
        field: 'author',
        type: 'nominal'
      }),
      y: Y({
        field: 'count',
        type: 'quantitative'
      })
    }),
    mark: 'bar',
    title: 'Top 50 commentors for binder-examples in the last 31 days',
    width: 1000
  }), Chart({
    data:              author  count authorAssociation
    2           Carreau     24            MEMBER
    14        blink1073     14            MEMBER
    58        terrdavis      6       CONTRIBUTOR
    21           daharn      6       CONTRIBUTOR
    16       ccordoba12      5            MEMBER
    46            minrk      4            MEMBER
    52         rmorshea      4      COLLABORATOR
    18         cshimmin      3              NONE
    48           palxex      3              NONE
    5             MSeal      3       CONTRIBUTOR
    59       texadactyl      3              NONE
    23      davidhalter      3              NONE
    57           syagev      2              NONE
    24           dlukes      2              NONE
    56         sthibaul      2              NONE
    8           Shaam93      2              NONE
    40      kevin-bates      2       CONTRIBUTOR
    61         willingc      2            MEMBER
    35            jaib1      2              NONE
    19      cyberwillis      2              NONE
    42          laoshaw      1              NONE
    45    matthew-brett      1              NONE
    44      lewisacidic      1              NONE
    43         letoncse      1              NONE
    41         krinsman      1       CONTRIBUTOR
    47            nouiz      1              NONE
    0     AbhinavBangia      1              NONE
    49       partha1968      1              NONE
    50         paulochf      1              NONE
    51    pikachust8811      1              NONE
    38         jrolfsen      1              NONE
    53           rsebbe      1              NONE
    54        silencial      1              NONE
    55         soxofaan      1              NONE
    60       vishakadsg      1              NONE
    62        yashk2810      1              NONE
    39       karan-gill      1              NONE
    32        gityou123      1              NONE
    37        jhayes-py      1              NONE
    36  jaideep11061982      1              NONE
    3     ChrisPalmerNZ      1              NONE
    4             Hubro      1              NONE
    6      MaximZemskov      1              NONE
    7        Miladiouss      1              NONE
    9       ShapingView      1              NONE
    10           Uiuran      1              NONE
    11         adam-m-c      1              NONE
    12           asishm      1              NONE
    13           banool      1              NONE
    15            bnaul      1              NONE,
    encoding: FacetedEncoding({
      color: Color({
        field: 'authorAssociation',
        scale: Scale({
          domain: ['MEMBER', 'CONTRIBUTOR', 'COLLABORATOR', 'NONE'],
          range: ['rgb(211.82745098039214, 211.82745098039214, 211.82745098039214, 256.0)', 'rgb(166.98562091503268, 229.89803921568628, 166.98562091503268, 256.0)', 'rgb(96.37647058823529, 192.75294117647059, 96.37647058823529, 256.0)', 'rgb(0.0, 100.3921568627451, 0.0, 256.0)']
        }),
        type: 'nominal'
      }),
      x: X({
        field: 'author',
        type: 'nominal'
      }),
      

## First responders

First responders are the first people to respond to a new issue in one of the repositories.
The following plots show first responders for recently-created issues.

In [30]:
first_comments = []
for (org, repo, issue_id), i_comments in comments.groupby(['org', 'repo', 'issue_id']):
    ix_min = pd.to_datetime(i_comments['createdAt']).idxmin()
    first_comment = i_comments.loc[ix_min]
    if isinstance(first_comment, pd.DataFrame):
        first_comment = first_comment.iloc[0]
    first_comments.append(first_comment)
first_comments = pd.concat(first_comments, axis=1).T

In [31]:
first_responder_counts = first_comments.groupby(['org', 'author', 'authorAssociation'], as_index=False).\
    count().rename(columns={'id': 'n_first_responses'}).sort_values(['org', 'n_first_responses'], ascending=False)


In [32]:
n_plot = 50
charts = []
for ii, (iorg, idata) in enumerate(first_responder_counts.groupby(['org'])):
    title = f"Top {n_plot} first responders for {iorg} in the last {n_days} days"
    idata = idata.groupby('author', as_index=False).agg({'n_first_responses': 'sum', 'authorAssociation': 'first'})
    idata = idata.sort_values('n_first_responses', ascending=False).head(n_plot)
    ch = alt.Chart(data=idata.head(n_plot), width=1000, title=title).mark_bar().encode(
        x='author',
        y='n_first_responses',
        color=alt.Color('authorAssociation', scale=alt.Scale(domain=author_types, range=author_colors))
    )
    charts.append(ch)
alt.hconcat(*charts)

HConcatChart({
  hconcat: [Chart({
    data:       author  n_first_responses authorAssociation
    0  choldgraf                  1            MEMBER,
    encoding: FacetedEncoding({
      color: Color({
        field: 'authorAssociation',
        scale: Scale({
          domain: ['MEMBER', 'CONTRIBUTOR', 'COLLABORATOR', 'NONE'],
          range: ['rgb(211.82745098039214, 211.82745098039214, 211.82745098039214, 256.0)', 'rgb(166.98562091503268, 229.89803921568628, 166.98562091503268, 256.0)', 'rgb(96.37647058823529, 192.75294117647059, 96.37647058823529, 256.0)', 'rgb(0.0, 100.3921568627451, 0.0, 256.0)']
        }),
        type: 'nominal'
      }),
      x: X({
        field: 'author',
        type: 'nominal'
      }),
      y: Y({
        field: 'n_first_responses',
        type: 'quantitative'
      })
    }),
    mark: 'bar',
    title: 'Top 50 first responders for binder-examples in the last 31 days',
    width: 1000
  }), Chart({
    data:             author  n_first_responses authorAssociation
    2          Carreau                 13            MEMBER
    31           minrk                  3            MEMBER
    13       blink1073                  3            MEMBER
    33          palxex                  2              NONE
    5            MSeal                  2       CONTRIBUTOR
    32           nouiz                  1              NONE
    25       jhayes-py                  1              NONE
    26        jrolfsen                  1              NONE
    27      karan-gill                  1              NONE
    28     kevin-bates                  1       CONTRIBUTOR
    29        krinsman                  1       CONTRIBUTOR
    30        letoncse                  1              NONE
    0    AbhinavBangia                  1              NONE
    23        gpotter2                  1       CONTRIBUTOR
    34      partha1968                  1              NONE
    35        paulochf                  1              NONE
    36        rmorshea                  1      COLLABORATOR
    37        soxofaan                  1              NONE
    38      texadactyl                  1              NONE
    39      vishakadsg                  1              NONE
    40        willingc                  1            MEMBER
    24    gustavosaume                  1              NONE
    21  farisachugthai                  1              NONE
    22    freedomtowin                  1              NONE
    11        adam-m-c                  1              NONE
    3    ChrisPalmerNZ                  1              NONE
    4            Hubro                  1              NONE
    6     MaximZemskov                  1              NONE
    7       Miladiouss                  1              NONE
    8          Shaam93                  1              NONE
    9      ShapingView                  1              NONE
    10          Uiuran                  1              NONE
    12          asishm                  1              NONE
    1         AnupBS28                  1              NONE
    14      ccordoba12                  1            MEMBER
    15       cycomanic                  1              NONE
    16          daharn                  1       CONTRIBUTOR
    17     davidhalter                  1              NONE
    18          dlukes                  1              NONE
    19       dragoljub                  1              NONE
    20     epogrebnyak                  1              NONE
    41       yashk2810                  1              NONE,
    encoding: FacetedEncoding({
      color: Color({
        field: 'authorAssociation',
        scale: Scale({
          domain: ['MEMBER', 'CONTRIBUTOR', 'COLLABORATOR', 'NONE'],
          range: ['rgb(211.82745098039214, 211.82745098039214, 211.82745098039214, 256.0)', 'rgb(166.98562091503268, 229.89803921568628, 166.98562091503268, 256.0)', 'rgb(96.37647058823529, 192.75294117647059, 96.37647058823529, 256.0)', 'rgb(0.0, 100.3921568627451, 0.0, 256.0)']
 

In [33]:
%%html
<script src="https://cdn.rawgit.com/parente/4c3e6936d0d7a46fd071/raw/65b816fb9bdd3c28b4ddf3af602bfd6015486383/code_toggle.js"></script>